In [49]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from preprocessing_pipeline import create_preprocessing_pipeline
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score

# Data Loading
Load the dataset and show first few rows. 

In [50]:
# Load the dataset
file_path = './marketing_campaign.csv'
df = pd.read_csv(file_path, sep='\t')
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


# Define Features (X)
[x]

In [51]:
# Define the features
X = df.columns

X

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response'],
      dtype='object')

# Split Data into Train and Test Sets
[x]

In [52]:
# Split the data into train and test sets

X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Create a Master Pipeline
 Chain data preprocessing and model steps together in a single object to ensure that all steps are applied sequentially and that data leakage is avoided.

In [53]:
# Create the pipeline with preprocessing and DBSCAN

master_pipeline = Pipeline(steps=[
    ('preprocessing', create_preprocessing_pipeline()),
    ('dbscan', DBSCAN(eps=0.5, min_samples=2))
])

master_pipeline

Pipeline(steps=[('preprocessing',
                 Pipeline(steps=[('feature_engineering',
                                  FunctionTransformer(func=<function create_preprocessing_pipeline.<locals>.<lambda> at 0x000001FB8F94A8B0>)),
                                 ('preprocessing',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('num',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer(strategy='median')),
                                                                                   ('scaler',
                                                                                    StandardScaler())]),
                                                                   ['...
                                                                    'NumWebPurchases',
                                                                    'NumCatalogPurchases',
                                                                    'NumStorePurchases',
                                                                    'NumWebVisitsMonth',
                                                                    'Z_CostContact',
                                                                    'Z_Revenue',
                                                                    'Age',
                                                                    'Total_Children',
                                                                    'Total_Household_Size',
                                                                    'Total_Spending',
                                                                    'Loyalty_Months',
                                                                    'Campaign_Response_Rate',
                                                                    'Shopping_Frequency']),
                                                                  ('cat',
                                                                   OneHotEncoder(handle_unknown='ignore'),
                                                                   ['Education',
                                                                    'Marital_Status_Grouped'])]))])),
                ('dbscan', DBSCAN(min_samples=2))])

# Fit Master Pipeline
[x]

In [54]:
# Fir the pipeline to the training data

master_pipeline.fit(X_train)

AttributeError: 'Index' object has no attribute 'assign'

# Predict Clusters on Both Train and Test Dats
[x]

In [ ]:
# Assign clusters to both train and test data

train_labels = master_pipeline.named_steps['dbscan'].labels_
test_labels = master_pipeline.named_steps['dbscan'].fit_predict(X_test)

# Add Cluster Labels to DataFrame
[x]

In [ ]:
# Add the cluster labels to the original DataFrame for visualization or analysis

X_train['Cluster'] = train_labels
X_test['Cluster'] = test_labels

# Display the results

print("Training Data with Clusters:")
print(X_train)

print("\nTest Data with Clusters:")
print(X_test)

# Calculate Silouette Score
[x]

In [ ]:
# Calculate the Silhouette Score for the test set (if there are more than 1 cluster)


if len(set(test_labels)) > 1:
    silhouette = silhouette_score(preprocessor.fit_transform(X_test), test_labels)
    print(f'\nSilhouette Score (Test Set): {silhouette}')
else:
    print('\nSilhouette Score (Test Set): Not applicable (only one cluster or all points are outliers)')

# Plot the Clusters
[x]

In [ ]:
# Plot the training set
plt.scatter(X_train['total_spend'], X_train['purchase_frequency'], c=train_labels, cmap='rainbow')
plt.title('DBSCAN Clustering - Train Set')
plt.xlabel('Total Spend')
plt.ylabel('Purchase Frequency')
plt.show()

In [ ]:
# Plot the test set
plt.scatter(X_test['total_spend'], X_test['purchase_frequency'], c=test_labels, cmap='rainbow')
plt.title('DBSCAN Clustering - Test Set')
plt.xlabel('Total Spend')
plt.ylabel('Purchase Frequency')
plt.show()

# Tune hyperparameters and find clusters
- Use elbow method to find k value
- Run the pipeline with the optimal value of k 
- Label the clusters and visualize it